In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from sklearn import model_selection
from sklearn import metrics
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:

dftrain=pd.read_json('../input/train.json')


In [ ]:
# first rows of df
dftrain.head()

In [ ]:
#dftrain['features'].apply(pd.Series)
allfeature=dftrain['features'].tolist()
sum(allfeature,[])

In [ ]:
# number of bathrooms with bedrooms
dftrain.groupby(['bathrooms','bedrooms']).count()

In [ ]:
# features columns count and flatten
dffeatures=dftrain['features'].apply(pd.Series)
dffstack=dffeatures.stack()
dffstack.value_counts()
feature_values=dffstack.values
feature_unique=list(set(feature_values))
print (feature_unique)

In [ ]:
len(feature_unique)

In [ ]:
# add feature for photo number and feature number
dftrain['photo number']=dftrain['photos'].apply(lambda x:len(x))
dftrain['feature number']=dftrain['features'].apply(lambda x:len(x))


In [ ]:
# replace target value with number
dftrain['interest_level'].replace(['low','medium','high'],[0,1,2],inplace=True)

In [ ]:
# define a value for bed/bath
def bedbath(bedno,bathno):
    if bathno != 0:
        number=bedno/bathno
    else:
        number=-1
    return number
    

In [ ]:
# add feature bed/bath ratio
dftrain['bedbathration']=dftrain.apply(lambda row: bedbath(row['bedrooms'],row['bathrooms']),axis=1)

In [ ]:
# prepare dataset for training
dftrain_use=dftrain.drop(['building_id','created','description','features','display_address','listing_id','photos','street_address'],axis=1)


In [ ]:
dftrain_use=dftrain_use.reset_index()
dftrain_use=dftrain_use.drop('index',axis=1)

In [ ]:
# cv for parameter search
kf=model_selection.KFold(n_splits=5)
for train_index,test_index in kf.split(dftrain_use):
    train,test=dftrain_use.iloc[train_index],dftrain_use.iloc[test_index]
    print (train_index)